In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import cluster
from scipy.cluster.vq import kmeans
import numpy as np
from pyod.models.knn import KNN

%matplotlib inline

In [20]:
df = pd.read_csv('test.csv', header=0, sep=';')

In [22]:
df['Time End Local Tz'] = pd.to_datetime(df['Time End Local Tz'], errors='coerce')
df['Time Start Local Tz'] = pd.to_datetime(df['Time Start Local Tz'], errors='coerce')

In [23]:
df.index = df['Time End Local Tz']
df = df.resample('D').mean()

In [25]:
df = df.fillna(df.mean())

In [26]:
train = df[0:79]
test = df[79:]

In [27]:
register_matplotlib_converters()

In [69]:
def find_anomalies(id=2166):
    df1 = df[df['User ID'] == id]
    value_raw = df1.Value.values
    value_raw = value_raw.reshape(-1, 1)
    value_raw = value_raw.astype('float64')
    X = df['Value'].values.reshape(-1,1)
    clf = KNN(contamination=0.038, n_neighbors=5)
    clf.fit(X)
    y_train_pred = clf.labels_ 
    y_train_scores = clf.decision_scores_
    arr = clf.predict(X)
    df1['Anomaly'] = arr
    return df1

In [70]:
res = find_anomalies()
res

,User ID,Value,Anomaly
Time End Local Tz,,,
2019-05-02 00:00:00+03:00,2166.0,60.924883,0
2019-05-03 00:00:00+03:00,2166.0,81.354067,0
2019-05-04 00:00:00+03:00,2166.0,72.649746,0
2019-05-05 00:00:00+03:00,2166.0,44.496000,0
2019-05-06 00:00:00+03:00,2166.0,49.780000,0
...,...,...,...
2019-08-10 00:00:00+03:00,2166.0,59.598958,0
2019-08-11 00:00:00+03:00,2166.0,80.009709,0
2019-08-12 00:00:00+03:00,2166.0,90.636364,0
